<a href="https://colab.research.google.com/github/mikhailgaerlan/TV-Show-Script-Writer/blob/master/Load_Spongebob_Script_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TV Show Script Generator

Generating scripts from a `gpt-2-simple` model finetuned on Spongebob Squarepants television show scripts scraped from Transcripts Wiki

by [Mikhail Gaerlan](https://www.math.ucdavis.edu/~mgaerlan)

Last updated: November 5, 2020

Based on work by [Max Woolf](http://minimaxir.com)

Please check out the original copy first [here](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwijivWvwunsAhVGlVkKHQIkDG4QFjAAegQIBhAC&url=https%3A%2F%2Fcolab.research.google.com%2Fdrive%2F1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce&usg=AOvVaw3W9t1DOZAaccv3zapLUhge).

## Installation and Version Requirements

GPT 2 Simple Requires Tensorflow 1.14/1.15 which is not supported above Python 3.8

In [1]:
from platform import python_version
print(python_version())

3.6.9


In [2]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [4]:
!nvidia-smi

Thu Nov  5 05:06:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

In [5]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 671Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 106Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 414Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:08, 60.6Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 321Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 135Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 194Mit/s]                                                       


## Mounting Google Drive

In [6]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [7]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [8]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-2000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [9]:
gpt2.generate(sess,
              length=1024,
              seed=511999,
              temperature=0.7,
              nsamples=1,
              batch_size=1,
              prefix="[Spongebob is watching TV while Gary is playing with a rock.] ",
              include_prefix=True
              )

[Spongebob is watching TV while Gary is playing with a rock.]  [screams] Gary!  [screams] [Gary is playing with an umbrella and SpongeBob is watching from above. SpongeBob and Gary are sitting in the picnic area, waiting for something.]  [screams] Gary! [Gary is playing with an umbrella] Hey, Gary, wait up! [SpongeBob and Gary are pushed to the edge of the area. SpongeBob slides down, hits a button and goes back up. They see a huge, wide-screened, red giant monster with a green shirt and square eyes. SpongeBob takes off his shirt and puts it over his square eyes, thinking it looks like the giant hand puppet. But, in reality, it was actually just a guard worm.] Giant Hand Pies: Oh, yeah, thats the most realistic explanation I could think of. [laughs]  [screams] [The giant hand puppet looks horrified. The guard worm slithers down towards SpongeBob and grumbles in disgust. A part of SpongeBobs eye lands on the giant hand puppet eye. SpongeBob giggles and slithers off. He looks over and se

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2020 Mikhail Gaerlan

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.